In [40]:
# data : https://www.smard.de/home/downloadcenter/download-marktdaten#!?downloadAttributes=%7B%22selectedCategory%22:3,%22selectedSubCategory%22:8,%22selectedRegion%22:%22DE%22,%22from%22:1658613600000,%22to%22:1659563999999,%22selectedFileType%22:%22CSV%22%7D

In [41]:
import pandas as pd
import pickle


In [42]:
df_smard = pd.read_csv('data/elect_price_2022.csv', delimiter=';', decimal=',', parse_dates=[['Datum', 'Uhrzeit']])
df_smard.tail()

,Datum_Uhrzeit,Deutschland/Luxemburg[€/MWh],∅ Anrainer DE/LU[€/MWh],Belgien[€/MWh],Dänemark 1[€/MWh],Dänemark 2[€/MWh],Frankreich[€/MWh],Niederlande[€/MWh],Norwegen 2[€/MWh],Österreich[€/MWh],Polen[€/MWh],Schweden 4[€/MWh],Schweiz[€/MWh],Tschechien[€/MWh],DE/AT/LU[€/MWh],Italien (Nord)[€/MWh],Slowenien[€/MWh],Ungarn[€/MWh]
7291,2022-10-31 19:00:00,167.81,"166,59",168.00,167.81,167.81,"168,22",167.96,162.95,165.64,166.39,162.95,"166,87",167.90,-,"173,62",173.62,"169,06"
7292,2022-10-31 20:00:00,147.09,"146,51",147.20,147.09,147.09,"147,3",147.17,142.72,145.97,147.01,142.72,"150,19",147.14,-,"149,99",149.99,"147,73"
7293,2022-10-31 21:00:00,136.78,"136,31",136.78,136.78,136.78,"136,78",136.78,132.81,136.78,136.78,132.81,"139,59",136.78,-,"136,78",136.78,"136,78"
7294,2022-10-31 22:00:00,120.51,"121,29",120.36,120.51,120.51,"120,85",119.90,116.77,124.23,134.39,101.16,"130,63",124.88,-,"125,13",125.13,"126,99"
7295,2022-10-31 23:00:00,100.73,-,100.42,100.73,100.73,"101,3",100.00,100.73,106.07,114.74,44.24,-,105.59,-,"115,63",106.92,"108,08"


In [43]:
ser_energy_price_germany = df_smard[['Datum_Uhrzeit', 'Deutschland/Luxemburg[€/MWh]']]
print(f"Type: {type(ser_energy_price_germany)}")
ser_energy_price_germany.head()

Type: <class 'pandas.core.frame.DataFrame'>


,Datum_Uhrzeit,Deutschland/Luxemburg[€/MWh]
0,2022-01-01 00:00:00,50.05
1,2022-01-01 01:00:00,41.33
2,2022-01-01 02:00:00,43.22
3,2022-01-01 03:00:00,45.46
4,2022-01-01 04:00:00,37.67


In [44]:
type(ser_energy_price_germany['Datum_Uhrzeit'])

pandas.core.series.Series

In [45]:
mask = ser_energy_price_germany['Datum_Uhrzeit'].duplicated()

print(ser_energy_price_germany[mask])

           Datum_Uhrzeit  Deutschland/Luxemburg[€/MWh]
7250 2022-10-30 02:00:00                         99.92


In [46]:
ser_energy_price_germany = ser_energy_price_germany.drop_duplicates(subset=['Datum_Uhrzeit'])

In [47]:
idx = ser_energy_price_germany['Datum_Uhrzeit']
val = ser_energy_price_germany['Deutschland/Luxemburg[€/MWh]']
ser = val.set_axis(idx, axis=0)

In [48]:
ser.head()

Datum_Uhrzeit
2022-01-01 00:00:00    50.05
2022-01-01 01:00:00    41.33
2022-01-01 02:00:00    43.22
2022-01-01 03:00:00    45.46
2022-01-01 04:00:00    37.67
Name: Deutschland/Luxemburg[€/MWh], dtype: float64

In [49]:
upsamled_ser = ser.resample('60s').pad()
#TODO: to datetime?? search for resample

/var/folders/2r/90tp356168b7h1_rljqp94f00000gn/T/ipykernel_11234/1912934808.py:1: FutureWarning: pad is deprecated and will be removed in a future version. Use ffill instead.
  upsamled_ser = ser.resample('60s').pad()


In [50]:
upsamled_ser.tail()

Datum_Uhrzeit
2022-12-10 22:56:00    243.55
2022-12-10 22:57:00    243.55
2022-12-10 22:58:00    243.55
2022-12-10 22:59:00    243.55
2022-12-10 23:00:00    228.57
Freq: 60S, Name: Deutschland/Luxemburg[€/MWh], dtype: float64

In [51]:
energy_price_lst = upsamled_ser.to_list()

with open('data/elect_price_2022.pkl', 'wb') as outfile:
    pickle.dump(energy_price_lst, outfile)

In [52]:
print(len(energy_price_lst))



495301
